In [41]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from bsf_dbutilities import DBUtils
from pyspark.sql import functions as F
import pandas as pd

try:
    spark.stop()
except NameError:
    # SparkSession doesn't exist
    pass

spark = init_spark("bsf_utilities", log_level="WARN", show_progress=False, enable_ui=True,process_option='manual')
engine = init_mariadb_engine()
ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

[Spark] Started 'bsf_utilities' log_level=WARN (effective=WARN), progress=False


25/10/04 09:49:09 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/10/04 09:49:09 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/10/04 09:49:09 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.


In [ ]:
# ─── Setup Database Communications ──────────────────────────────────────
db = DBUtils(spark, ingest_ts)
db.spark_stats()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand width to avoid line breaks


from delta.tables import DeltaTable
import datetime, os
from pyspark.sql import functions as F

# List all databases
databases = [db.name for db in spark.catalog.listDatabases()]

for database in databases:
    print(f"\n=== Database: {database} ===\n")
    
    # Get all tables in this database
    tables = spark.catalog.listTables(database)
    if not tables:
        print("No tables found.")
        continue

    table_info = []
    
    for t in tables:
        full_table_name = f"{database}.{t.name}"
        row_count = spark.table(full_table_name).count()
        table_type = t.tableType
    
        # Default: no max date
        from pyspark.sql import functions as F
        import datetime
        import os
        
        max_date = None
        try:
            df_table = spark.table(full_table_name)
            
            if "LastLoadedDate" in df_table.columns:
                max_date = df_table.agg(F.max("LastLoadedDate")).collect()[0][0]
                field = "LastLoadedDate"
            elif "StockDate" in df_table.columns:
                max_date = df_table.agg(F.max("StockDate")).collect()[0][0]
                field = "StockDate"
            elif "ChangeDate" in df_table.columns:
                max_date = df_table.agg(F.max("ChangeDate")).collect()[0][0]
                field = "ChangeDate"
            else:
                try:
                    dt = DeltaTable.forName(spark, full_table_name)
                    history = dt.history(1)
                    max_date = history.select(F.max("timestamp")).collect()[0][0]
                    field = "timestamp"
                except:
                    max_date = None
        
            # --- Normalize to datetime ---
            if isinstance(max_date, datetime.date) and not isinstance(max_date, datetime.datetime):
                max_date = datetime.datetime.combine(max_date, datetime.time.min)
        
        except:
            # Non-Delta fallback
            table_location = (
                spark.sql(f"DESCRIBE FORMATTED {full_table_name}")
                     .filter("col_name='Location'")
                     .select("data_type")
                     .collect()[0][0]
            )
            max_date = datetime.datetime.fromtimestamp(os.path.getmtime(table_location))


    
        table_info.append((t.name, row_count, field, max_date, table_type))
    
    df_table_info = spark.createDataFrame(
        table_info,
        schema=["TableName", "RowCount","DateField", "MaxDate", "TableType"]
    )
    
    df_table_info.show(truncate=False)
    # 🚨 Safety stop — prevents accidental full execution
    raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")

In [ ]:
spark.stop()

In [ ]:
sdf = spark.table("bsf.company")
sdf.toPandas().describe()


In [ ]:
from copy import deepcopy
from bsf_config import CONFIG

In [ ]:
def load_settings(profile: str = "default" ):
    """Load settings, merging defaults with optional profile overrides.
       For `timeframe_map`, profile overrides REPLACE the whole dict
       instead of merging.
    """
    default_settings = deepcopy(CONFIG["default"])
    settings = deepcopy(default_settings)

    if profile and profile in CONFIG:
        overrides = CONFIG[profile]

        def merge(base, update, path=""):
            for k, v in update.items():
                current_path = f"{path}.{k}" if path else k

                # Special case: timeframe_map is replace, not merge
                if k == "timeframe_map":
                    old_value = base.get(k, "<not in default>")
                    #print(f"Override: {current_path}: default={old_value}, profile={v}")
                    base[k] = deepcopy(v)
                    continue

                if isinstance(v, dict) and k in base and isinstance(base[k], dict):
                    merge(base[k], v, current_path)
                else:
                    old_value = base.get(k, "<not in default>")
                    #print(f"Override: {current_path}: default={old_value}, profile={v}")
                    base[k] = deepcopy(v)

        merge(settings, overrides)

    return settings


In [ ]:
import json
mbsetting = load_settings()

config_json = json.dumps(mbsetting)
print(config_json)

In [ ]:
import json
mbsetting = load_settings("tier1")

config_json = json.dumps(mbsetting)
print(config_json)

In [86]:
wm_df = spark.sql(f"""
    SELECT *
    FROM bsf.companystockhistory_watermark
""").toPandas()
wm_dict = dict(zip(wm_df.CompanyId, wm_df.LastLoadedDate))
print(wm_df)

   CompanyId LastLoadedDate
0         52     2025-10-03


In [62]:
from pyspark.sql import functions as F

# Assuming your stock history table is a Delta table
df = spark.table("bsf.companystockhistory_watermark")

# Count rows per company per stock date
df.groupBy("CompanyId", "StockDate") \
  .agg(F.count("*").alias("row_count")) \
  .filter("row_count > 1") \
  .orderBy("CompanyId", "StockDate") \
  .show(50, truncate=False)


+---------+---------+---------+
|CompanyId|StockDate|row_count|
+---------+---------+---------+
+---------+---------+---------+



#############################################

In [74]:
df = spark.table("bsf.companystockhistory_watermark")
print(df.count())
df.show()

1
+---------+--------------+
|CompanyId|LastLoadedDate|
+---------+--------------+
|       52|    2025-10-03|
+---------+--------------+



In [87]:
spark.table("bsf.companystockhistory") \
     .orderBy(F.col("StockDate").desc()) \
     .show(10, truncate=False)

+---------+----------+-------+-------+-------+-------+--------+
|CompanyId|StockDate |Open   |High   |Low    |Close  |Volume  |
+---------+----------+-------+-------+-------+-------+--------+
|52       |2025-10-03|0.04549|0.04549|0.041  |0.041  |108000.0|
|52       |2025-10-02|0.044  |0.044  |0.044  |0.044  |0.0     |
|52       |2025-10-01|0.04005|0.044  |0.04005|0.044  |50000.0 |
|52       |2025-09-30|0.044  |0.044  |0.044  |0.044  |9900.0  |
|52       |2025-09-29|0.0441 |0.0441 |0.037  |0.04349|156505.0|
|52       |2025-09-26|0.054  |0.054  |0.054  |0.054  |0.0     |
|52       |2025-09-25|0.054  |0.054  |0.054  |0.054  |0.0     |
|52       |2025-09-24|0.0449 |0.054  |0.0449 |0.054  |10066.0 |
|52       |2025-09-23|0.035  |0.035  |0.035  |0.035  |0.0     |
|52       |2025-09-22|0.051  |0.055  |0.035  |0.035  |271066.0|
+---------+----------+-------+-------+-------+-------+--------+
only showing top 10 rows



In [ ]:
spark.table("bsf.companystockhistory") \
     .orderBy(F.col("StockDate").desc()) \
     .show(10, truncate=False)

In [88]:
df = spark.table("bsf.companyfundamental")
print(df.count())
df.show()

21
+---------+---------------+-------+--------+--------+--------------+--------------+------------------+-----------------+------------+----------------+-------------+-------------+--------+---------------+
|CompanyId|FundamentalDate|PeRatio|PegRatio| PbRatio|ReturnOnEquity|GrossMarginTTM|NetProfitMarginTTM|TotalDebtToEquity|CurrentRatio|InterestCoverage|EpsChangeYear|RevChangeYear|    Beta|ShortIntToFloat|
+---------+---------------+-------+--------+--------+--------------+--------------+------------------+-----------------+------------+----------------+-------------+-------------+--------+---------------+
|       52|     2024-12-29| -630.0|     0.0|-684.211|           0.0|           0.0|               0.0|              0.0|         0.4|             0.0|          0.0|          0.0|-3.76912|            0.0|
|       52|     2025-01-05| -800.0|     0.0|-684.211|           0.0|           0.0|               0.0|              0.0|         0.4|             0.0|          0.0|          0.0|-3.

In [71]:
df = spark.table("bsf.companyfundamental")
print(df.count())
df.show()

21
+---------+---------------+-------+--------+--------+--------------+--------------+------------------+-----------------+------------+----------------+-------------+-------------+--------+---------------+
|CompanyId|FundamentalDate|PeRatio|PegRatio| PbRatio|ReturnOnEquity|GrossMarginTTM|NetProfitMarginTTM|TotalDebtToEquity|CurrentRatio|InterestCoverage|EpsChangeYear|RevChangeYear|    Beta|ShortIntToFloat|
+---------+---------------+-------+--------+--------+--------------+--------------+------------------+-----------------+------------+----------------+-------------+-------------+--------+---------------+
|       52|     2024-11-24| -760.0|     0.0|-684.211|           0.0|           0.0|               0.0|              0.0|         0.4|             0.0|          0.0|          0.0|-3.75422|            0.0|
|       52|     2024-12-01| -530.0|     0.0|-684.211|           0.0|           0.0|               0.0|              0.0|         0.4|             0.0|          0.0|          0.0|-3.

In [84]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# Reference the table
table_name = "bsf.companystockhistory_watermark"
dt = DeltaTable.forName(spark, table_name)

# Update CompanyId = 52 to new date
dt.update(
    condition = F.col("CompanyId") == 52,
    set = { "LastLoadedDate": F.lit("2025-10-01") }
)


In [ ]:
from pyspark.sql import functions as F

sdf = spark.table("bsf.companystockhistory_watermark")

# Count rows per timeframe
sdf.groupBy("CompanyId") \
   .agg(F.max("LastLoadedDate").alias("max_date")) \
   .orderBy("CompanyId") \
   .show(truncate=False)

        watermark_update_df = sdf.groupBy("CompanyId") \
                               .agg(F.max("StockDate").alias("LastLoadedDate"))

In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.vacuum(retentionHours=0)  # ⚠ deletes unreferenced files immediately


In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.history().show(truncate=False)



In [ ]:
spark.read.format("delta").load("/srv/lakehouse/tables/bsf.db/history_signals") \
     .select("TimeFrame").distinct().show()



In [ ]:
count = spark.sql(f"""
    SELECT COUNT(DISTINCT CompanyId) as cnt
    FROM bsf.company
    WHERE ListingExchange IN (1,2,3,16)
      AND Active = 1
      AND LastClose < 0.1
      AND LastHistoryDate >= date_sub(current_date(), 30)
""").collect()[0]["cnt"]

print(count)




In [ ]:
sdf = spark.table("bsf.companystockhistory")
sdf.toPandas().describe()

In [ ]:
sdf = spark.table("bsf.companystockhistory")
counts_df = (
    sdf.groupBy("CompanyId")
      .agg(F.count("*").alias("row_count"))
      .orderBy("CompanyId")
)

counts_df.show(50, truncate=False)
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.companystockhistory_watermark")
pdf = sdf.limit(10).toPandas()
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.history_signals_last")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.signaldriver")
sdf.toPandas().describe()



In [ ]:
sdf = spark.table("bsf.history_signals")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.final_candidates")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.company")
# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy("ListingExchange")
      .agg(F.count("*").alias("row_count"))
      .orderBy("ListingExchange")
)

counts_df.show(50, truncate=False)



sdf = spark.table("bsf.history_signals")

# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy( "TimeFrame")
      .agg(F.count("*").alias("row_count"))
      .orderBy( "TimeFrame")
)

counts_df.show(50, truncate=False)


In [ ]:
# Assume you have a Spark DataFrame
sdf = spark.table("bsf.companystockhistory")

# 1️⃣ Show the schema (data types included)
sdf.printSchema()

# 2️⃣ Get a list of column names
# print(sdf.columns)

# 3️⃣ Show first few rows with all columns (default shows truncated view)
sdf.show(truncate=False)  

# 4️⃣ For more detailed metadata about columns
sdf.dtypes  # Returns a list of (column_name, data_type)


In [ ]:
# 🚨 Safety stop — prevents accidental full execution
raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")
